In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

#import data

fname = '../Train/Train/X_station_train.csv'
df = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
d_sub_station = df[df['number_sta'] == 14066001 ]
#preview of time data for 1 station
display(d_sub_station.head())
display(d_sub_station.tail())

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4


,number_sta,date,ff,t,td,hu,dd,precip,Id
4403495,14066001,2017-12-30 19:00:00,9.63,285.38,283.40,87.7,233.0,0.2,14066001_729_19
4403496,14066001,2017-12-30 20:00:00,9.80,285.70,283.16,84.6,230.0,0.0,14066001_729_20
4403497,14066001,2017-12-30 21:00:00,10.67,286.07,282.83,80.8,230.0,0.0,14066001_729_21
4403498,14066001,2017-12-30 22:00:00,10.02,286.53,283.01,79.4,230.0,0.0,14066001_729_22
4403499,14066001,2017-12-30 23:00:00,10.67,286.84,283.22,78.9,230.0,0.0,14066001_729_23


ValueError: This function does not handle missing values

Process data to handle rows with missing values

In [ ]:
data_precip = d_sub_station['precip']

#
# decomposition = seasonal_decompose(data_precip, model='multiplicative')
# trend    = decomposition.trend
# seasonal = decomposition.seasonal
# residual = decomposition.resid